In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets, models, transforms
from torchvision import models
import os

import numpy as np
import matplotlib.pyplot as plt
import time
import copy

from sklearn.metrics import confusion_matrix

from torch.utils.data.sampler import  WeightedRandomSampler

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
model_path = "./drive/My Drive/Colab Notebooks/"

In [0]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=False, num_workers=2)

In [0]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [0]:
device

In [0]:
dataloaders = {'train': trainloader, 'val': testloader}
dataset_sizes = {'train': len(trainset), 'val': len(testset)}

In [0]:
res_net = models.resnet50(pretrained=False)
num_ftrs = res_net.fc.in_features
res_net.fc = nn.Linear(num_ftrs, 10)
PATH = model_path+"/rotation_net.pth"
res_net.load_state_dict(torch.load(PATH))
res_net = res_net.to(device)
res_net.eval()
matrix = [[727.,  22.,  64.,  32.,  11.,  10.,  17.,   8.,  84.,  25.],
        [ 46., 789.,  18.,  11.,   3.,   1.,  13.,   3.,  30.,  86.],
        [ 89.,   1., 542.,  79., 104.,  63.,  87.,  21.,   7.,   7.],
        [ 23.,   4.,  66., 554.,  59., 142., 105.,  30.,   7.,  10.],
        [ 31.,   4.,  69.,  64., 595.,  39.,  89.,  97.,   8.,   4.],
        [ 12.,   0.,  50., 211.,  47., 594.,  36.,  45.,   1.,   4.],
        [ 19.,   5.,  51.,  67.,  38.,  19., 797.,   2.,   1.,   1.],
        [ 21.,   2.,  56.,  65.,  69.,  56.,  20., 689.,   6.,  16.],
        [105.,  50.,  21.,  16.,   6.,   1.,   6.,   7., 743.,  45.],
        [ 42.,  95.,  13.,  17.,   8.,   3.,  10.,  16.,  37., 759.]]
matrix = np.array(matrix)/1000

In [0]:
res_net2 = models.resnet50(pretrained=False)
num_ftrs = res_net2.fc.in_features
res_net2.fc = nn.Linear(num_ftrs, 10)
PATH = model_path+"rotation_imagenet_net.pth"
res_net2.load_state_dict(torch.load(PATH))
res_net2 = res_net2.to(device)
res_net2.eval()
matrix2 = [[838.,  10.,  31.,  14.,   6.,   3.,   3.,   5.,  70.,  20.],
        [ 24., 854.,   9.,   3.,   3.,   3.,   7.,   1.,  24.,  72.],
        [ 72.,   4., 669.,  32.,  72.,  59.,  50.,  33.,   9.,   0.],
        [ 33.,   2.,  75., 645.,  45., 118.,  41.,  27.,   8.,   6.],
        [ 21.,   1.,  75.,  56., 708.,  26.,  46.,  55.,   9.,   3.],
        [  6.,   0.,  54., 135.,  38., 689.,  19.,  55.,   2.,   2.],
        [ 12.,   2.,  61.,  46.,  37.,  23., 814.,   1.,   2.,   2.],
        [ 20.,   2.,  54.,  45.,  81.,  49.,   6., 725.,   3.,  15.],
        [104.,  27.,   8.,   3.,   5.,   0.,   3.,   0., 834.,  16.],
        [ 34.,  70.,   3.,   6.,   2.,   6.,   1.,   8.,  34., 836.]]
matrix2 = np.array(matrix2)/1000


In [0]:
res_net3 = models.resnet50(pretrained=False)
num_ftrs = res_net3.fc.in_features
res_net3.fc = nn.Linear(num_ftrs, 10)
PATH = model_path+"resnet_net.pth"
res_net3.load_state_dict(torch.load(PATH))
res_net3 = res_net3.to(device)
res_net3.eval()
matrix3 = [[448.,  74.,  54.,  34.,  30.,  25.,  24., 102., 175.,  34.],
        [ 76., 457.,  33.,  29.,  23.,  38.,  39., 178.,  47.,  80.],
        [118.,  42., 188.,  43., 192.,  54., 121., 180.,  44.,  18.],
        [ 46.,  48.,  79., 168., 103., 129., 135., 255.,  24.,  13.],
        [ 53.,  26.,  82.,  39., 321.,  38., 169., 236.,  25.,  11.],
        [ 45.,  35.,  64., 117., 105., 219., 105., 263.,  28.,  19.],
        [ 23.,  43.,  92.,  43., 135.,  49., 374., 225.,  10.,   6.],
        [ 59.,  50.,  33.,  46.,  83.,  49.,  80., 547.,  19.,  34.],
        [250.,  97.,  39.,  21.,  25.,  21.,  25.,  94., 393.,  35.],
        [121., 251.,  31.,  17.,  16.,  35.,  21., 201.,  79., 228.]]
matrix3 = np.array(matrix3)/1000


In [0]:
res_net4 = models.resnet50(pretrained=True)
num_ftrs = res_net4.fc.in_features
res_net4.fc = nn.Linear(num_ftrs, 10)
PATH = model_path+"resnet_pretrained.pth"
res_net4.load_state_dict(torch.load(PATH))
res_net4 = res_net4.to(device)
res_net4.eval()
matrix4 = [[856.,  13.,  23.,   9.,   6.,   1.,   2.,  12.,  63.,  15.],
        [ 30., 860.,   3.,   8.,   3.,   0.,   1.,   5.,  25.,  65.],
        [ 25.,   1., 795.,  29.,  52.,  22.,  54.,  11.,   9.,   2.],
        [ 10.,   1.,  38., 744.,  30.,  98.,  39.,  26.,   8.,   6.],
        [ 14.,   0.,  45.,  27., 815.,  17.,  26.,  44.,   8.,   4.],
        [  3.,   2.,  15., 115.,  22., 788.,  14.,  36.,   3.,   2.],
        [  7.,   1.,  29.,  26.,  30.,   9., 892.,   3.,   3.,   0.],
        [ 10.,   0.,  22.,  17.,  41.,  27.,   5., 872.,   1.,   5.],
        [ 48.,  12.,   4.,   7.,   3.,   1.,   2.,   0., 918.,   5.],
        [ 18.,  39.,   4.,  10.,   3.,   1.,   1.,   5.,  22., 897.]]
matrix4 = np.array(matrix4)/1000

In [0]:
res_net5 = models.resnet50(pretrained=False)
num_ftrs = res_net5.fc.in_features
res_net5.fc = nn.Linear(num_ftrs, 10)
PATH = model_path+"relative_pos_net.pth"
res_net5.load_state_dict(torch.load(PATH))
res_net5 = res_net5.to(device)
res_net5.eval()
matrix5 = [[470.,  55.,  39.,   4.,  33.,  52.,  56.,  12., 230.,  49.],
        [111., 438.,   1.,   2.,  29.,  37., 101.,  33., 101., 147.],
        [143.,  25., 168.,   7., 133., 205., 195.,  23.,  80.,  21.],
        [ 66.,  40.,  42.,  29.,  92., 358., 226.,  39.,  45.,  63.],
        [ 32.,  41.,  26.,   4., 388., 156., 229.,  47.,  48.,  29.],
        [ 36.,  29.,  42.,  23.,  99., 498., 175.,  45.,  21.,  32.],
        [ 62.,  31.,  23.,   4.,  95., 105., 585.,  19.,  40.,  36.],
        [ 57.,  67.,  15.,   9., 136., 184., 118., 245.,  45., 124.],
        [274., 111.,  14.,   1.,  37.,  29.,  59.,  17., 385.,  73.],
        [128., 153.,   2.,   8.,  28.,  44.,  85.,  62.,  98., 392.]]
matrix5 = np.array(matrix5)/1000

In [0]:
res_net6 = models.resnet50(pretrained=False)
num_ftrs = res_net6.fc.in_features
res_net6.fc = nn.Linear(num_ftrs, 10)
PATH = model_path+"relative_pos_imagenet_net.pth"
res_net6.load_state_dict(torch.load(PATH))
res_net6 = res_net6.to(device)
res_net6.eval()
matrix6 = [[420.,  61.,  89.,  22.,  50.,  20.,  48.,  31., 201.,  58.],
        [ 46., 488.,   5.,  33.,  44.,  15.,  99.,  91.,  71., 108.],
        [ 90.,  23., 252.,  54., 207.,  58., 183.,  59.,  60.,  14.],
        [ 51.,  86.,  91., 141., 135., 115., 210., 109.,  30.,  32.],
        [ 23.,  44.,  78.,  43., 414.,  48., 205.,  96.,  38.,  11.],
        [ 33.,  53.,  85., 152., 149., 193., 164., 122.,  26.,  23.],
        [ 34.,  61.,  64.,  51., 131.,  28., 535.,  50.,  17.,  29.],
        [ 37.,  92.,  45.,  61., 124.,  53., 113., 362.,  36.,  77.],
        [189., 107.,  57.,  15.,  60.,  10.,  50.,  49., 406.,  57.],
        [ 51., 256.,  11.,  24.,  26.,   8.,  59., 125.,  89., 351.]]
matrix6 = np.array(matrix6)/1000

In [0]:
res_net7 = models.resnet50(pretrained=False)
num_ftrs = res_net7.fc.in_features
res_net7.fc = nn.Linear(num_ftrs, 10)
PATH = model_path+"jigsaw_net.pth"
res_net7.load_state_dict(torch.load(PATH))
res_net7 = res_net7.to(device)
res_net7.eval()
matrix7 = [[616.,  71.,  38.,  25.,  43.,  16.,  13.,  30.,  88.,  60.],
        [ 64., 636.,   2.,  32.,  31.,   6.,  40.,  28.,  19., 142.],
        [118.,  34., 246.,  90., 245.,  67.,  91.,  67.,  15.,  27.],
        [ 57.,  59.,  61., 263., 172., 131.,  93., 103.,   9.,  52.],
        [ 30.,  39.,  45.,  58., 546.,  39., 124.,  85.,  12.,  22.],
        [ 25.,  18.,  51., 193., 164., 337.,  54., 108.,  10.,  40.],
        [ 31.,  80.,  38.,  55., 211.,  17., 475.,  44.,   6.,  43.],
        [ 34.,  71.,  19.,  60., 135.,  71.,  39., 423.,   9., 139.],
        [276., 150.,  34.,  20.,  27.,   5.,  17.,  18., 362.,  91.],
        [ 66., 192.,   7.,  22.,  45.,  12.,  22.,  53.,  18., 563.]]
matrix7 = np.array(matrix7)/1000

In [0]:
res_net8 = models.resnet50(pretrained=False)
num_ftrs = res_net8.fc.in_features
res_net8.fc = nn.Linear(num_ftrs, 10)
PATH = model_path+"jigsaw_imagenet_net.pth"
res_net8.load_state_dict(torch.load(PATH))
res_net8 = res_net8.to(device)
res_net8.eval()
matrix8 = [[745.,  22.,  45.,  32.,   9.,   7.,  14.,  15.,  90.,  21.],
        [ 10., 834.,   3.,  16.,   0.,   1.,  15.,   3.,  33.,  85.],
        [ 46.,   0., 637.,  72.,  60.,  41., 101.,  26.,  12.,   5.],
        [ 15.,   6.,  41., 638.,  25., 126.,  93.,  37.,   9.,  10.],
        [ 13.,   2., 105.,  86., 547.,  33., 111.,  86.,  16.,   1.],
        [  7.,   2.,  37., 205.,  23., 623.,  32.,  61.,   7.,   3.],
        [  2.,   4.,  46.,  62.,  15.,  15., 844.,   3.,   9.,   0.],
        [  8.,   3.,  29.,  57.,  27.,  41.,  24., 801.,   8.,   2.],
        [ 35.,  21.,  12.,  12.,   2.,   3.,   7.,   5., 887.,  16.],
        [ 16.,  75.,   9.,  17.,   1.,   4.,   7.,  10.,  36., 825.]]
matrix8 = np.array(matrix8)/1000

In [0]:
confusion_matrix = torch.zeros(len(classes), len(classes))
right = 0
all = 0
pre = [] 
tru = []
for inputs, labels in dataloaders['val']:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = res_net(inputs)
    outputs2 = res_net2(inputs)
    outputs3 = res_net3(inputs)
    outputs4 = res_net4(inputs)
    outputs5 = res_net5(inputs)
    outputs6 = res_net6(inputs)
    outputs7 = res_net7(inputs)
    outputs8 = res_net8(inputs)
    _, preds = torch.max(outputs, 1)
    _, preds2 = torch.max(outputs2, 1)
    _, preds3 = torch.max(outputs3, 1)
    _, preds4 = torch.max(outputs4, 1)
    _, preds5 = torch.max(outputs5, 1)
    _, preds6 = torch.max(outputs6, 1)
    _, preds7 = torch.max(outputs7, 1)
    _, preds8 = torch.max(outputs8, 1)
    for t, p1, p2, p3, p4, p5, p6, p7, p8 in zip(labels.view(-1), preds.view(-1), preds2.view(-1), preds3.view(-1), preds4.view(-1), preds5.view(-1), preds6.view(-1), preds7.view(-1), preds8.view(-1)):
        vote = torch.zeros(len(classes))
        for i in range(0,10):
          vote[i]+=1.0/np.e**matrix[p1.long()][p1.long()]*np.e**matrix[i][p1.long()]
          vote[i]+=1.0/np.e**matrix[p2.long()][p2.long()]*np.e**matrix[i][p2.long()]
          vote[i]+=1.0/np.e**matrix[p3.long()][p3.long()]*np.e**matrix[i][p3.long()]
          vote[i]+=1.0/np.e**matrix[p4.long()][p4.long()]*np.e**matrix[i][p4.long()]
          vote[i]+=1.0/np.e**matrix[p5.long()][p5.long()]*np.e**matrix[i][p5.long()]
          vote[i]+=1.0/np.e**matrix[p6.long()][p6.long()]*np.e**matrix[i][p6.long()]
          vote[i]+=1.0/np.e**matrix[p7.long()][p7.long()]*np.e**matrix[i][p7.long()]
          vote[i]+=1.0/np.e**matrix[p8.long()][p8.long()]*np.e**matrix[i][p8.long()]
        a = vote.numpy()
        b = np.where(a==np.max(a))
        c = torch.from_numpy(b[0])[0]
        tru.append(t.long().cpu())
        pre.append(c.long().cpu())
        confusion_matrix[t.long(), c.long()] += 1
        if(t.long() == c.long()):
          right+=1
        all+=1

print(confusion_matrix)
print(right/all)
from sklearn.metrics import f1_score, precision_score, recall_score
f1 = f1_score( tru, pre, average='macro' )
print(f1)

In [0]:
confusion_matrix = torch.zeros(len(classes), len(classes))
right = 0
all = 0
pre = []
tru = []
for inputs, labels in dataloaders['val']:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = res_net(inputs)
    outputs2 = res_net2(inputs)
    outputs3 = res_net3(inputs)
    outputs4 = res_net4(inputs)
    outputs5 = res_net5(inputs)
    outputs6 = res_net6(inputs)
    outputs7 = res_net7(inputs)
    outputs8 = res_net8(inputs)
    _, preds = torch.max(outputs, 1)
    _, preds2 = torch.max(outputs2, 1)
    _, preds3 = torch.max(outputs3, 1)
    _, preds4 = torch.max(outputs4, 1)
    _, preds5 = torch.max(outputs5, 1)
    _, preds6 = torch.max(outputs6, 1)
    _, preds7 = torch.max(outputs7, 1)
    _, preds8 = torch.max(outputs8, 1)
    for t, p1, p2, p3, p4, p5, p6, p7, p8 in zip(labels.view(-1), preds.view(-1), preds2.view(-1), preds3.view(-1), preds4.view(-1), preds5.view(-1), preds6.view(-1), preds7.view(-1), preds8.view(-1)):
        vote = torch.zeros(len(classes))
        vote[p1.long()]+=1
        vote[p2.long()]+=1
        vote[p3.long()]+=1
        vote[p4.long()]+=1
        vote[p5.long()]+=1
        vote[p6.long()]+=1
        vote[p7.long()]+=1
        vote[p8.long()]+=1
        a = vote.numpy()
        b = np.where(a==np.max(a))
        if(len(b[0]) != 1):
          x1 = matrix[p1.long()][p1.long()]
          x2 = matrix2[p2.long()][p2.long()]
          x3 = matrix3[p3.long()][p3.long()]
          x4 = matrix4[p4.long()][p4.long()]
          x5 = matrix5[p5.long()][p5.long()]
          x6 = matrix6[p6.long()][p6.long()]
          x7 = matrix7[p7.long()][p7.long()]
          x8 = matrix8[p8.long()][p8.long()]
          if(x1 == max(x1,x2,x3,x4,x5,x6,x7,x8)):
            c = p1
          elif(x2 == max(x1,x2,x3,x4,x5,x6,x7,x8)):
            c = p2
          elif(x3 == max(x1,x2,x3,x4,x5,x6,x7,x8)):
            c = p3
          elif(x4 == max(x1,x2,x3,x4,x5,x6,x7,x8)):
            c = p4
          elif(x5 == max(x1,x2,x3,x4,x5,x6,x7,x8)):
            c = p5
          elif(x6 == max(x1,x2,x3,x4,x5,x6,x7,x8)):
            c = p6
          elif(x7 == max(x1,x2,x3,x4,x5,x6,x7,x8)):
            c = p7
          elif(x8 == max(x1,x2,x3,x4,x5,x6,x7,x8)):
            c = p8
        else:
          c = torch.from_numpy(b[0])[0]
        confusion_matrix[t.long(), c.long()] += 1
        pre.append(c.long().cpu())
        tru.append(t.long().cpu())
        all+=1
        if(t.long() == c.long()):
          right+=1

print(confusion_matrix)
print(right/all)
from sklearn.metrics import f1_score, precision_score, recall_score
f1 = f1_score( tru, pre, average='macro' )
print(f1)